# Atmospheric deposition dataset extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas=2.1.3
* scipy=1.9.0
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* 


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 


## References
* 
## Observations
* 

# Import modules

In [1]:
import pandas as pd
import numpy as np
import tqdm as tqdm
import os
import glob
import warnings
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib as mpl
import glob
from matplotlib.lines import Line2D
import matplotlib as mpl
import rasterio


c:\Users\nascimth\AppData\Local\miniconda3\envs\geopandas\lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


# Configurations

In [2]:
# Only editable variables:
# Relative path to your local directory
PATH = ".."
# Suppress all warnings
warnings.filterwarnings("ignore")

* #### The users should NOT change anything in the code below here. 

In [3]:
# Non-editable variables:
PATH_OUTPUT = "results/atmospheric_deposition/"

# Set the directory:
os.chdir(PATH)

In [4]:
os.getcwd()

'c:\\Users\\nascimth\\Documents\\Thiago\\Eawag\\Python\\Scripts\\CAMELS_CH_chem'

# Import data
* Full table

In [36]:
#catchments = gpd.read_file('data/shps/basins_187.shp')
#catchments["bafu_id"] = catchments["gauge_id"]
#catchments

In [5]:
catchments = gpd.read_file('data/shps/CAMELS_CH_Chem_catchments.shp')
catchments

,gauge_id,ID6,gauge_name,water_body,type,country,Shape_Leng,Shape_Area,ORIG_FID,geometry
0,2009.0,RhoPor,Porte_du_Scex,Rhône,stream,CH,581812.194190,5.239402e+09,2,"POLYGON Z ((2674253.038 1167429.881 0.000, 267..."
1,2011.0,RhoSio,Sion,Rhône,stream,CH,373714.667313,3.372417e+09,3,"POLYGON Z ((2674253.038 1167429.881 0.000, 267..."
2,2016.0,AarBru,Brugg,Aare,stream,CH,827842.590414,1.168128e+10,5,"POLYGON Z ((2655969.680 1259695.589 0.000, 265..."
3,2018.0,ReuMel,Mellingen,Reuss,stream,CH,423447.043212,3.385753e+09,7,"POLYGON Z ((2663723.380 1252919.068 0.000, 266..."
4,2019.0,AarBri,Brienzwiler,Aare,stream,CH,135675.028325,5.551571e+08,8,"POLYGON Z ((2669196.412 1183579.510 0.000, 266..."
...,...,...,...,...,...,...,...,...,...,...
110,2617.0,RomMue,Müstair,Rom,stream,CH,58967.655299,1.285851e+08,218,"POLYGON Z ((2820942.826 1171469.984 0.000, 282..."
111,2623.0,RhoObe,Oberwald,Rhone,stream,CH,57228.199696,9.331790e+07,220,"POLYGON Z ((2674253.038 1167429.881 0.000, 267..."
112,2634.0,KlEEmm,Emmen,Kleine_Emme,stream,CH,142185.158427,4.782772e+08,224,"POLYGON Z ((2653429.237 1216261.807 0.000, 265..."
113,2635.0,GroEin,Einsiedeln-Gross,Grossbach,stream,CH,13804.039981,8.945090e+06,225,"POLYGON Z ((2701144.527 1218073.633 0.000, 270..."


In [6]:
# Network CAMELS_CH_Chem
network_camels_ch_chem = pd.read_excel(r"data/CAMELS_CH_chem_stations_short_v3.xlsx", sheet_name='all_5')
#network_camels_ch_chem.set_index("basin_id", inplace=True)
network_camels_ch_chem

,basin_id,bafu_id,naduf_id,nawa_id,isot_id,hydro_station,hydrowater_body,lon,lat,area_camels,...,lon_naduf,lat_naduf,area_naduf,bafu_naduf_distance,nawa_station,lon_nawa,lat_nawa,area_nawa,bafu_nawa_distance,remarks.1
0,2009,2009.0,1837.0,1837.0,NIO04,Porte du Scex,Rhône,557660,133280,5239.4,...,557660.0,133280.0,5239.4,0.0,Porte du Scex,557660.0,133280.0,5239.402096,0.0,NaN
1,2011,2011.0,NaN,4070.0,NaN,Sion,Rhône,593770,118630,3372.4,...,NaN,NaN,NaN,NaN,Sion,593277.0,118449.0,3372.417040,0.0,NaN
2,2016,2016.0,1833.0,1833.0,NIO02,Brugg,Aare,657000,259360,11681.3,...,657000.0,259360.0,11681.3,0.0,Brugg,657000.0,259360.0,11681.282882,0.0,NaN
3,2018,2018.0,1835.0,1339.0,NaN,Mellingen,Reuss,662830,252580,3385.8,...,662830.0,252580.0,3385.8,0.0,Gebenstorf,659450.0,258850.0,3420.503458,10.0,NaN
4,2019,2019.0,NaN,1852.0,NIO01,Brienzwiler,Aare,649930,177380,555.2,...,NaN,NaN,NaN,NaN,Brienzerseeeinlauf,646692.0,177000.0,555.808970,3.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,2617,2617.0,NaN,NaN,NaN,Müstair,Rom,830800,168700,128.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,2623,2623.0,NaN,NaN,NaN,Oberwald,Rhone,669900,154075,93.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,2634,2634.0,6169.0,1181.0,NaN,Emmen,Kleine Emme,663700,213630,478.3,...,663700.0,213630.0,478.3,0.0,Emmen-Littau,663917.0,213356.0,478.277165,0.6,NaN
113,2635,2635.0,NaN,NaN,NaN,"Einsiedeln, Gross",Grossbach,700710,218125,8.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# The majority is primarly formed by the bafu id, so here we check the oens that ARE NOT:
network_camels_ch_chem[network_camels_ch_chem.bafu_id.isna()]

,basin_id,bafu_id,naduf_id,nawa_id,isot_id,hydro_station,hydrowater_body,lon,lat,area_camels,...,lon_naduf,lat_naduf,area_naduf,bafu_naduf_distance,nawa_station,lon_nawa,lat_nawa,area_nawa,bafu_nawa_distance,remarks.1
10,2053,NaN,NaN,3860.0,NaN,"Martigny, Pont de Rossettan",Drance,570930,105200,675.7,...,NaN,NaN,NaN,NaN,Martigny,570930.0,105200.0,675.654142,0.0,NaN
12,2063,NaN,NaN,1108.0,NaN,Murgenthal,Aare,629665,235150,10059.4,...,NaN,NaN,NaN,NaN,Murgenthal,629760.0,235242.0,10059.355199,0.3,NaN
19,2099,NaN,NaN,1053.0,NaN,"Zürich, Unterhard",Limmat,682055,249430,2173.9,...,NaN,NaN,NaN,NaN,Hönggersteg,679609.0,250356.0,2186.067125,2.7,NaN
20,2102,NaN,NaN,5023.0,NaN,Sarnen,Sarner Aa,661460,194220,269.3,...,NaN,NaN,NaN,NaN,"Sarnen, Kägiswil",662600.0,195770.0,284.246739,2.1,NaN
26,2122,NaN,NaN,1494.0,NaN,"Moutier, La Charrue",Birse,595740,237010,185.8,...,NaN,NaN,NaN,NaN,Choindez- Aval de la Roche St-Jean,596293.0,240496.0,198.704397,3.9,NaN
27,2125,NaN,NaN,1265.0,NaN,Frauenthal,Lorze,674715,229845,261.7,...,NaN,NaN,NaN,NaN,Frauenthal,674725.0,229850.0,261.723315,0.0,NaN
35,2155,NaN,NaN,6315.0,NaN,"Wiler, Limpachmündung",Emme,608220,223240,924.1,...,NaN,NaN,NaN,NaN,"Gerlafingen, Brücke nach Biberist",609527.0,225566.0,924.578661,3.3,Q from id_0155
37,2160,NaN,NaN,2025.0,NaN,"Broc, Château d'en bas",Sarine,573520,161345,636.2,...,NaN,NaN,NaN,NaN,"Broc, Halte des Marches",573520.0,161345.0,636.248285,0.0,NaN
45,2202,NaN,NaN,1307.0,NaN,Liestal,Ergolz,622270,259750,261.2,...,NaN,NaN,NaN,NaN,"Augst, Autobahn",620950.0,264930.0,284.647126,5.9,NaN
46,2203,NaN,NaN,2115.0,NaN,Aigle,Grande Eau,563960,129860,131.6,...,NaN,NaN,NaN,NaN,"Aigle, Autoroute",561112.0,130518.0,144.375806,3.0,NaN


In [28]:
## Merge the DataFrames for achieving the bafu_id ias the last column
#catchments = pd.merge(catchments, network_camels_ch_chem[['bafu_id', 'basin_id']], on='bafu_id', how='left')
#catchments = pd.merge(catchments, network_camels_ch_chem[['nawa_id', 'basin_id']], on='nawa_id', how='left')
#catchments = pd.merge(catchments, network_camels_ch_chem[['naduf_id', 'basin_id']], on='naduf_id', how='left')
#
#catchments

In [29]:
## Replace all 0s with NaN:
#catchments.loc[:, ['gauge_id', "naduf_id", "nawa_id"]] = catchments.loc[:, ['gauge_id', "naduf_id", "nawa_id"]].replace(0, np.nan)

In [30]:
## Create the new 'basin_id' column based on the priority order
#catchments['basin_id_new'] = np.nan
#catchments['basin_id_new'] = np.where(
#    catchments['gauge_id'].notna(), catchments['basin_id_x'],
#    np.where(
#        catchments['nawa_id'].notna(), catchments['basin_id_y'],
#        catchments['basin_id']
#    )
#)
#
## Display the updated DataFrame
#catchments

In [35]:
## Here we can check the data
#catchments[catc£hments.basin_id_new.isna()]

In [31]:
## Here we solve it manuallly
#catchments.loc[84, ["basin_id_new"]] = 2403.0
#catchments.loc[110, ["basin_id_new"]] = 2622.0
#
#catchments[catchments.basin_id_new.isna()]

In [32]:
## Drop duplicates based on 'gauge_id'
#catchments_unique = catchments.drop_duplicates(subset="basin_id_new")
#catchments_unique.set_index("basin_id_new", inplace=True)
#catchments_unique

In [33]:
## Delete the 2403 (BAFU) since it is empty for our time range, and 2243 (merged with 2622):
#catchments_unique.drop(2403.0, axis=0, inplace=True)
#catchments_unique.drop(2243.0, axis=0, inplace=True)


In [34]:
#catchments_unique

In [8]:
from collections import defaultdict

# List of filenames
filenames_rasters = [
    'data/Ndep_cog500_241105_Teil1_Teil2/dhno3gas_1990.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2000.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2005.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2010.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2015.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2020.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh3gas_1990.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2000.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2005.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2010.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2015.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2020.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh4total_1990.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh4total_2000.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh4total_2005.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh4total_2010.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh4total_2015.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dnh4total_2020.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno2gas_1990.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno2gas_2000.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno2gas_2005.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno2gas_2010.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno2gas_2015.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno2gas_2020.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno3total_1990.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno3total_2000.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno3total_2005.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno3total_2010.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno3total_2015.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dno3total_2020.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dntotal_1990.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dntotal_2000.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dntotal_2005.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dntotal_2010.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dntotal_2015.tif',
    'data/Ndep_cog500_241105_Teil1_Teil2/dntotal_2020.tif'
]

# Group files by their prefixes
file_groups = defaultdict(list)

for filename in filenames_rasters:
    prefix = filename.split("/")[-1].split("_")[0]  # Extract prefix (e.g., 'dhno3gas')
    file_groups[prefix].append(filename)

In [9]:
file_groups.keys()

dict_keys(['dhno3gas', 'dnh3gas', 'dnh4total', 'dno2gas', 'dno3total', 'dntotal'])

## Reproject to projected coordinates system


In [ ]:
## Define the target CRS to ETRS89 LAEA
#target_crs = 'EPSG:2056'
#
## Reproject the GeoDataFrame to the target CRS
#catchments_unique_reprojected = catchments_unique.to_crs(target_crs)

In [10]:
# Define the target CRS to ETRS89 LAEA
target_crs = 'EPSG:2056'

# Reproject the GeoDataFrame to the target CRS
catchments_unique_reprojected = catchments.to_crs(target_crs)

In [11]:
# Set the index and adjust to int (instead of float)
catchments_unique_reprojected.set_index("gauge_id", inplace=True)
catchments_unique_reprojected.index = catchments_unique_reprojected.index.astype(int)

## Computation processes


In [19]:
# Convert DataFrame to GeoDataFrame with valid geometries
catchments_unique_reprojected = gpd.GeoDataFrame(
    catchments_unique_reprojected,
    geometry=catchments_unique_reprojected['geometry'],  # Use the existing geometry column
    crs="EPSG:2056"  # Adjust CRS as necessary
)

In [41]:
from tqdm import tqdm
import pandas as pd
import rasterio
import numpy as np
from rasterio.mask import geometry_mask
from shapely.geometry import mapping
from rasterio.features import geometry_mask

# Define prefixes for their names based on the order of lecture
prefix_values = ["1990_", "2000_", "2005_", "2010_", "2015_", "2020_"]

compounds = ['dhno3gas', 'dnh3gas', 'dnh4total', 'dno2gas', 'dno3total', 'dntotal']

# Iterate over each unique code
for code in tqdm(catchments_unique_reprojected.index):
    # Initialize an empty DataFrame to store the results for the current code
    code_df = pd.DataFrame()

    for compound in compounds:
        # Initialize a DataFrame for the current compound
        compound_data = []
        compound_df = pd.DataFrame()

        i = 0
        filenames = file_groups[compound]

        for filename in filenames:
            # Create lists to store the results
            avg_values = []

            # Load your raster file
            with rasterio.open(filename) as src:
                geom = catchments_unique_reprojected.loc[[code]]

                # Check if the geometry is empty or invalid
                geometry = geom['geometry'].iloc[0]

                if geometry is None or geometry.is_empty or not geometry.is_valid:
                    avg_value = np.nan
                else:
                    
                    # Convert geometry to GeoJSON-like dict for rasterio
                    geom_mapping = [mapping(geometry)]
                    
                    # Create a mask for the geometry
                    mask = geometry_mask(geom_mapping, out_shape=src.shape, transform=src.transform, invert=True)

                    # Read the values within the geometry from the raster
                    values = src.read(1, masked=True)
                    values = values[mask]

                # Calculate statistics only if there are valid values in the 'values' array
                if len(values) > 0:
                    avg_value = np.sum(values)
                else:
                    # Handle the case when there are no valid values
                    avg_value = np.nan

                # Append the result for the current year (prefix)
                compound_data.append(avg_value)

            i += 1

        # Add the compound data as a column to the code DataFrame
        code_df[compound] = compound_data

    # Add year prefixes as the index
    code_df.index = [prefix[:-1] for prefix in prefix_values]

    code_df.index.name = "date"

    code_df.to_csv(PATH_OUTPUT + "/CAMELS_CH_Chem_atm_"+str(code)+".csv", encoding='latin')



100%|██████████| 115/115 [02:35<00:00,  1.35s/it]


# End